# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tindouf,27.6711,-8.1474,18.25,25,0,3.60,DZ,1701288699
1,1,bethel,41.3712,-73.4140,2.90,52,0,4.12,US,1701288558
2,2,olonkinbyen,70.9221,-8.7187,-1.29,93,97,7.49,SJ,1701288699
3,3,bairnsdale,-37.8333,147.6167,14.47,99,100,2.24,AU,1701288700
4,4,hailar,49.2000,119.7000,-25.62,94,5,2.80,CN,1701288700


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,tolanaro,-25.0319,46.9987,24.68,76,0,2.14,MG,1701288730
237,237,zouerate,22.7187,-12.4521,21.43,11,0,3.05,MR,1701288773
276,276,vallenar,-28.5708,-70.7581,25.88,23,0,3.76,CL,1701288785
431,431,darwin,-12.4611,130.8418,26.99,89,0,0.00,AU,1701288827
483,483,gebeit,21.0667,36.3167,21.40,78,0,2.96,SD,1701288839
543,543,as suwayq,23.8494,57.4386,24.75,66,0,1.05,OM,1701288859


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Humidity': humidity
}
hotel_df=pd.DataFrame(hotel_dict)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
91,tolanaro,MG,-25.0319,46.9987,76,
237,zouerate,MR,22.7187,-12.4521,11,
276,vallenar,CL,-28.5708,-70.7581,23,
431,darwin,AU,-12.4611,130.8418,89,
483,gebeit,SD,21.0667,36.3167,78,
543,as suwayq,OM,23.8494,57.4386,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
zouerate - nearest hotel: فندق تيرس زمور
vallenar - nearest hotel: Hotel del Marqués
darwin - nearest hotel: Mantra Pantanas Darwin
gebeit - nearest hotel: No hotel found
as suwayq - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
91,tolanaro,MG,-25.0319,46.9987,76,Hôtel Mahavokey
237,zouerate,MR,22.7187,-12.4521,11,فندق تيرس زمور
276,vallenar,CL,-28.5708,-70.7581,23,Hotel del Marqués
431,darwin,AU,-12.4611,130.8418,89,Mantra Pantanas Darwin
483,gebeit,SD,21.0667,36.3167,78,No hotel found
543,as suwayq,OM,23.8494,57.4386,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)